In [1]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
import matplotlib.pyplot as plt

from scipy.spatial.distance import pdist, cdist, squareform
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics.pairwise import laplacian_kernel,chi2_kernel
###############################################################################
# Fit regression model
#svr1 = SVR(kernel='rbf', C=100, gamma=0.1)
#svr1=SVR()
#svr2 = SVR(kernel=mymahal, C=100, gamma='auto')
#svr3 = SVR(kernel=myminkowski, C=100)
###############################################################################
from sklearn.model_selection import GridSearchCV
import time

In [2]:
def rpdd(obs,pred):
    rmsep=mean_squared_error(pred,obs,squared=False)

    # ratio of performance to deviation
    rpd=np.std(obs) / rmsep
    return rpd

def nrmse(obs, rmse):
    NRMSE=rmse/(np.max(obs)-np.min(obs))
    return NRMSE

In [3]:
import pandas as pd
#filepath="G:\\E-backup\\study\\study\\2023\\hefeng\\dataprocess\\data2.xls"
filepath="data2.xls"
df_laser_tar=pd.read_excel(open(filepath, 'rb'), sheet_name='在线激光打孔参数与焦油量测定结果表')

tratio=(df_laser_tar['焦油量(不打孔)']-df_laser_tar['焦油量'])/df_laser_tar['焦油量(不打孔)']
z=tratio.dropna()
#scaler=StandardScaler()
#scaler2=StandardScaler()

In [4]:
oldz=z

scaler2=MinMaxScaler()
scaler=StandardScaler()
XX = np.column_stack((df_laser_tar['孔数/排'], df_laser_tar['时间']))
yy = np.array(z)
X = scaler.fit_transform(XX)
y = scaler.fit_transform(yy.reshape(-1, 1))

In [7]:
from sklearn.gaussian_process.kernels import StationaryKernelMixin,NormalizedKernelMixin,Kernel,Hyperparameter
from sklearn.base import clone
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.utils.validation import _num_samples

class RationalQuadratic_minkow_g(StationaryKernelMixin, NormalizedKernelMixin, Kernel):
    """Rational Quadratic kernel.

    The RationalQuadratic kernel can be seen as a scale mixture (an infinite
    sum) of RBF kernels with different characteristic length scales. It is
    parameterized by a length scale parameter :math:`l>0` and a scale
    mixture parameter :math:`\\alpha>0`. Only the isotropic variant
    where length_scale :math:`l` is a scalar is supported at the moment.
    The kernel is given by:

    .. math::
        k(x_i, x_j) = \\left(
        1 + \\frac{d(x_i, x_j)^2 }{ 2\\alpha  l^2}\\right)^{-\\alpha}

    where :math:`\\alpha` is the scale mixture parameter, :math:`l` is
    the length scale of the kernel and :math:`d(\\cdot,\\cdot)` is the
    Euclidean distance.
    For advice on how to set the parameters, see e.g. [1]_.

    Read more in the :ref:`User Guide <gp_kernels>`.

    .. versionadded:: 0.18

    Parameters
    ----------
    length_scale : float > 0, default=1.0
        The length scale of the kernel.

    alpha : float > 0, default=1.0
        Scale mixture parameter

    length_scale_bounds : pair of floats >= 0 or "fixed", default=(1e-5, 1e5)
        The lower and upper bound on 'length_scale'.
        If set to "fixed", 'length_scale' cannot be changed during
        hyperparameter tuning.

    alpha_bounds : pair of floats >= 0 or "fixed", default=(1e-5, 1e5)
        The lower and upper bound on 'alpha'.
        If set to "fixed", 'alpha' cannot be changed during
        hyperparameter tuning.

    References
    ----------
    .. [1] `David Duvenaud (2014). "The Kernel Cookbook:
        Advice on Covariance functions".
        <https://www.cs.toronto.edu/~duvenaud/cookbook/>`_

    Examples
    --------
    >>> from sklearn.datasets import load_iris
    >>> from sklearn.gaussian_process import GaussianProcessClassifier
    >>> from sklearn.gaussian_process.kernels import RationalQuadratic
    >>> X, y = load_iris(return_X_y=True)
    >>> kernel = RationalQuadratic(length_scale=1.0, alpha=1.5)
    >>> gpc = GaussianProcessClassifier(kernel=kernel,
    ...         random_state=0).fit(X, y)
    >>> gpc.score(X, y)
    0.9733...
    >>> gpc.predict_proba(X[:2,:])
    array([[0.8881..., 0.0566..., 0.05518...],
            [0.8678..., 0.0707... , 0.0614...]])
    """

    def __init__(
        self,
        length_scale=1.0,
        alpha=2.0,
        length_scale_bounds=(1e-5, 1e5),
        alpha_bounds=(1e-5, 1e5),
    ):
        self.length_scale = length_scale
        self.alpha = alpha
        self.length_scale_bounds = length_scale_bounds
        self.alpha_bounds = alpha_bounds

    @property
    def hyperparameter_length_scale(self):
        return Hyperparameter("length_scale", "numeric", self.length_scale_bounds)

    @property
    def hyperparameter_alpha(self):
        return Hyperparameter("alpha", "numeric", self.alpha_bounds)

 
    def __call__(self, X, Y=None, eval_gradient=False):
        """Return the kernel k(X, Y) and optionally its gradient.

        Parameters
        ----------
        X : ndarray of shape (n_samples_X, n_features)
            Left argument of the returned kernel k(X, Y)

        Y : ndarray of shape (n_samples_Y, n_features), default=None
            Right argument of the returned kernel k(X, Y). If None, k(X, X)
            if evaluated instead.

        eval_gradient : bool, default=False
            Determines whether the gradient with respect to the log of
            the kernel hyperparameter is computed.
            Only supported when Y is None.

        Returns
        -------
        K : ndarray of shape (n_samples_X, n_samples_Y)
            Kernel k(X, Y)

        K_gradient : ndarray of shape (n_samples_X, n_samples_X, n_dims)
            The gradient of the kernel k(X, X) with respect to the log of the
            hyperparameter of the kernel. Only returned when eval_gradient
            is True.
        """
        if len(np.atleast_1d(self.length_scale)) > 1:
            raise AttributeError(
                "RationalQuadratic kernel only supports isotropic version, "
                "please use a single scalar for length_scale"
            )
        X = np.atleast_2d(X)
        if Y is None:
            dists = squareform(pdist(X, metric="minkowski",p=5))
            tmp = dists / (2 * self.alpha * self.length_scale**2)
            base = 1 + tmp
            K = base**-self.alpha
            np.fill_diagonal(K, 1)
            print("No Y input")
        else:
            if eval_gradient:
                raise ValueError("Gradient can only be evaluated when Y is None.")
            dists = cdist(X, Y, metric="minkowski",p=5)
            K = (1 + dists / (2 * self.alpha * self.length_scale**2)) ** -self.alpha

        if eval_gradient:
            # gradient with respect to length_scale
            if not self.hyperparameter_length_scale.fixed:
                length_scale_gradient = dists * K / (self.length_scale**2 * base)
                length_scale_gradient = length_scale_gradient[:, :, np.newaxis]
            else:  # l is kept fixed
                length_scale_gradient = np.empty((K.shape[0], K.shape[1], 0))

            # gradient with respect to alpha
            if not self.hyperparameter_alpha.fixed:
                alpha_gradient = K * (
                    -self.alpha * np.log(base)
                    + dists / (2 * self.length_scale**2 * base)
                )
                alpha_gradient = alpha_gradient[:, :, np.newaxis]
            else:  # alpha is kept fixed
                alpha_gradient = np.empty((K.shape[0], K.shape[1], 0))

            return K, np.dstack((alpha_gradient, length_scale_gradient))
        else:
            return K

    def __repr__(self):
        return "{0}(alpha={1:.3g}, length_scale={2:.3g})".format(
            self.__class__.__name__, self.alpha, self.length_scale
        )


In [8]:

df_laser_7mg=pd.read_excel(open(filepath, 'rb'), sheet_name='焦油含量7 mg的在线激光打孔参数组合表')
df_laser_6mg=pd.read_excel(open(filepath, 'rb'), sheet_name='焦油含量6 mg的在线激光打孔参数组合表')
df_sensory_6mg=pd.read_excel(open(filepath, 'rb'), sheet_name='焦油含量6 mg卷烟感官评价结果表')

In [ ]:
import pandas as pd

pd.set_option("display.precision",10)
df=pd.DataFrame.from_dict(resultlist3)

In [ ]:
resultlist4=[]

from sklearn.model_selection import KFold

k=5
kf=KFold(n_splits=k,shuffle=True, random_state=42)

for train_index, test_index in kf.split(X):
    X_train, X_test=X.iloc[train_index], X.iloc[test_index]
    y_train, y_test=y.iloc[train_index],y.iloc[test_index]
    

# SVR with the grid search method

In [12]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
from sklearn.model_selection import GridSearchCV
parameters={'C':np.arange(1,201,5),'gamma':np.arange(1e-7,1,0.1),'epsilon':np.arange(0.01,1,0.1)}

resultslist4=[]

def RationalQuadratic2(x,y,length_scale=2,alpha=1):
    #length_scale = 2.0
    #alpha = 1.0
    dists = cdist(x, y, metric='sqeuclidean')
    K = (1 + dists / (2 * alpha * length_scale ** 2)) ** - alpha
    return K

def RationalQuadratic_minkow_p(x,y,pp=5,length_scale=2,alpha=1):
    #length_scale = 2.0
    #alpha = 2.0
    dists = cdist(x, y, metric='minkowski',p=pp)
    K = (1 + dists / (2 * alpha * length_scale ** 2)) ** - alpha
    return K

def myrbf3(x,y,length_scale):
    #length_scale=2
    dists = cdist(x / length_scale, y / length_scale,  metric='sqeuclidean')
    res = np.exp(-.5* dists)
    return res

svr_rbf = SVR(kernel='rbf',C=51, epsilon=0.1, gamma=0.5)
svr_lin = SVR(kernel='linear',C=51, epsilon=0.1,gamma=1e-07)
svr_rq=SVR(kernel=RationalQuadratic2,C=51, epsilon=0.1, gamma=1e-07)
svr_rm= SVR(kernel=RationalQuadratic_minkow_p,C=151, epsilon=0.1, gamma=1e-07)

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.gaussian_process.kernels import RationalQuadratic

kernel_lin=DotProduct()
kernel_rm = RationalQuadratic_minkow_g(length_scale=2,alpha=1)  # new class for gaussian kernel has been defined in previous section
kernel_rbf= RBF(length_scale=2)
kernel_rq=RationalQuadratic(length_scale=2,alpha=1)  
#kernel=RBF(length_scale=1.0)
gp_lin = GaussianProcessRegressor(kernel=kernel_lin, normalize_y=True)
gp_rq=GaussianProcessRegressor(kernel=kernel_rq, normalize_y=True)

gp_rm = GaussianProcessRegressor(kernel=kernel_rm, normalize_y=True)
gp_rbf=GaussianProcessRegressor(kernel=kernel_rbf, normalize_y=True)

#svr_mahalanobis= SVR(kernel=mymahal,C=201, epsilon=0.01, gamma=1e-07)
#svr_minkowski=SVR(kernel=myminkowski,C=201, epsilon=0.01, gamma=1e-07)

# #############################################################################
# Look at the results
lw = 2

#models = [svr_lin,svr_rbf,  svr_rq, svr_rm,gp_lin, gp_rbf,gp_rq,gp_rm]
models = ['linear','rbf',RationalQuadratic2,RationalQuadratic_minkow_p,gp_lin, gp_rbf,gp_rq,gp_rm]
#models = [RationalQuadratic2,RationalQuadratic_minkow_p]

kernel_label = ['SVM Linear', 'SVM RBF','SVM RQ', 'SVM RM','GP Linear','GP RBF','GP RQ','GP RM']
#kernel_label = ['SVM RQ', 'SVM RM']

model_color = ['m', 'c', 'g','r','y','#00FFFF','#8FBC8F','#A0522D']


In [13]:

#fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(12, 12), sharey=True)
for ix, model in enumerate(models):
        #model=svr.fit(X,y)
        #rn=range(len(y))
        #model.decision_function(X.ravel()) 

        out1=[]
        start=time.time()
        if 'SVM' in kernel_label[ix]:
            modeltmp= SVR(kernel=model)
            grid_search=GridSearchCV(modeltmp,parameters,cv=5)
            grid_search.fit(X,y)
            bestmodel=SVR(kernel=model,**grid_search.best_params_)
            model=bestmodel
        for i in range(0,df_laser_tar.shape[0]):
                gx=np.delete(X,i,0)
                gy=np.delete(y,i,0)
                testx = X[i,:].reshape(1,2)             
                o1 = model.fit(gx,gy.flatten()).predict(testx)
                out1.append(o1[0])
        end=time.time()
        mso1=mean_squared_error(y,np.array(out1).reshape(-1,1),squared=False)
        nrmse1=nrmse(y,mso1)
        rso1=r2_score(y,np.array(out1).reshape(-1,1))
        rpd1=rpdd(y,np.array(out1).reshape(-1,1))
        print('nrmse:'+str(nrmse1)+' o1 rmse:'+str(mso1)+'  rs2:'+str(rso1 )+'  rpd:'+str(rpd1))
        print('out1:'+str(out1))
        dd=dict(kernelname=kernel_label[ix],nrmse=nrmse1,rmse=mso1,r2=rso1,rpd=rpd1,time=(end-start),params=grid_search.best_params_)
        resultslist4.append(dd)

nrmse:0.1847286431230497 o1 rmse:0.6449792496625212  rs2:0.5840017675047711  rpd:1.5504374761253787
out1:[0.7351815255691511, 1.1508341773039885, 0.7349474642923866, 1.1508341773039885, 0.4193925884916044, 0.21094778153299765, 0.5138140861412013, 0.27617987493090185, -0.0032131190298433088, 0.03854566372061492, 0.7514482973514913, 0.8348207776740627, 1.0686003929545398, 0.7514482973514913, 1.0680480216391142, 0.910533316508367, -0.5175985930710691, -0.5175985930710691, -0.27996438186076983, -0.8336119927009675, -1.073742849862755, -1.073742849862755, -0.27996438186076983, -0.2774677359092887, -0.8336119927009675, -0.13684447586745133, -1.6298871066544378, -1.3897562494926503, -0.8361086386524597, -1.6298871066544378, -1.3897562494926503, -0.6716562028531735]
nrmse:0.08488125129823912 o1 rmse:0.2963625177297862  rs2:0.9121692580848622  rpd:3.3742458650313116
out1:[1.677770467256571, 1.629697757583565, 1.9349805326337552, 1.7645347825758688, 0.6251403111426996, 0.7476648237862236, 0.6251

In [54]:
grid_search.best_params_


{'C': 16, 'epsilon': 0.11, 'gamma': 1e-07}

In [14]:
import pandas as pd

pd.set_option("display.precision",10)
df=pd.DataFrame.from_dict(resultslist4)
df

,kernelname,nrmse,rmse,r2,rpd,time,params
0,SVM Linear,0.1847286431,0.6449792497,0.5840017675,1.5504374761,65.8655939102,"{'C': 11, 'epsilon': 0.7100000000000001, 'gamm..."
1,SVM RBF,0.0848812513,0.2963625177,0.9121692581,3.3742458650,28.0556702614,"{'C': 91, 'epsilon': 0.01, 'gamma': 0.10000010..."
2,SVM RQ,0.0974149151,0.3401237501,0.8843158346,2.9401063573,25.2676975727,"{'C': 96, 'epsilon': 0.31000000000000005, 'gam..."
3,SVM RM,0.0700368622,0.2445333981,0.9402034172,4.0894209445,24.9611752033,"{'C': 16, 'epsilon': 0.11, 'gamma': 1e-07}"
4,GP Linear,0.1905602274,0.6653402008,0.5573224172,1.5029904984,0.2092382908,"{'C': 16, 'epsilon': 0.11, 'gamma': 1e-07}"
5,GP RBF,0.0702262944,0.2451947998,0.9398795102,4.0783899204,0.0853378773,"{'C': 16, 'epsilon': 0.11, 'gamma': 1e-07}"
6,GP RQ,0.0702263049,0.2451948364,0.9398794922,4.0783893108,0.7057085037,"{'C': 16, 'epsilon': 0.11, 'gamma': 1e-07}"
7,GP RM,0.0702263243,0.2451949043,0.9398794589,4.0783881815,0.6127557755,"{'C': 16, 'epsilon': 0.11, 'gamma': 1e-07}"


In [58]:
import dill

dill.dump_session('svr_with_search_grid4.pkl')  # Save the entire session

# SVR with the L-BFGS-B method, not work well

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from scipy.optimize import fmin_l_bfgs_b

def objective(params):
    C, epsilon, gamma = params
    svr = SVR(C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    return mse

from scipy.optimize import fmin_l_bfgs_b

initial_guess = [1.0, 0.1, 0.1]  # Initial values for C, epsilon, and gamma
bounds = [(1e-3, 1e2), (1e-3, 1.0), (1e-3, 10)]  # Bounds for C, epsilon, and gamma

result = fmin_l_bfgs_b(objective, initial_guess, bounds=bounds)
best_C, best_epsilon, best_gamma = result[0]

best_svr = SVR(C=best_C, epsilon=best_epsilon, gamma=best_gamma)
best_svr.fit(X_train, y_train)

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from scipy.optimize import fmin_l_bfgs_b

def objective(params,kernelt):
    C, epsilon, gamma = params
    svr = SVR(kernel=kernelt, C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    return mse

lw = 2
from scipy.optimize import fmin_l_bfgs_b

initial_guess = [1.0, 0.1, 0.1]  # Initial values for C, epsilon, and gamma
bounds = [(1e-3, 1e2), (1e-3, 1.0), (1e-3, 10)]  # Bounds for C, epsilon, and gamma

models = ['linear','rbf',RationalQuadratic2,RationalQuadratic_minkow_p,gp_lin, gp_rbf,gp_rq,gp_rm]
kernel_label = ['SVM Linear', 'SVM RBF','SVM RQ', 'SVM RM','GP Linear','GP RBF','GP RQ','GP RM']
model_color = ['m', 'c', 'g','r','y','#00FFFF','#8FBC8F','#A0522D']

#svr_rbf = SVR(kernel='rbf',C=51, epsilon=0.1, gamma=0.5)
#svr_lin = SVR(kernel='linear',C=51, epsilon=0.1,gamma=1e-07)
#svr_rq=SVR(kernel=RationalQuadratic2,C=51, epsilon=0.1, gamma=1e-07)
#svr_rm= SVR(kernel=RationalQuadratic_minkow_p,C=151, epsilon=0.1, gamma=1e-07)
#fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(12, 12), sharey=True)
for ix, model in enumerate(models):
        #model=svr.fit(X,y)
        #rn=range(len(y))
        #model.decision_function(X.ravel()) 

        out1=[]
        start=time.time()
        if 'SVM' in kernel_label[ix]:
            result = fmin_l_bfgs_b(objective(kernelt=model), initial_guess, bounds=bounds)
            best_C, best_epsilon, best_gamma = result[0]
            bestmodel=SVR(kernel=model,C=best_C, epsilon=best_epsilon, gamma=best_gamma)
            modelt=bestmodel
        else:
            modelt=model   
        for i in range(0,df_laser_tar.shape[0]):
                gx=np.delete(X,i,0)
                gy=np.delete(y,i,0)
                testx = X[i,:].reshape(1,2)             
                o1 = modelt.fit(gx,gy.flatten()).predict(testx)
                out1.append(o1[0])
        end=time.time()
        mso1=mean_squared_error(y,np.array(out1).reshape(-1,1),squared=False)
        nrmse1=nrmse(y,mso1)
        rso1=r2_score(y,np.array(out1).reshape(-1,1))
        rpd1=rpdd(y,np.array(out1).reshape(-1,1))
        print('nrmse:'+str(nrmse1)+' o1 rmse:'+str(mso1)+'  rs2:'+str(rso1 )+'  rpd:'+str(rpd1))
        print('out1:'+str(out1))
        dd=dict(kernelname=kernel_label[ix],nrmse=nrmse1,rmse=mso1,r2=rso1,rpd=rpd1,time=(end-start))
        resultslist4.append(dd)